# Step 0

Importing a bunch of modules, there might be some redundancies.

In [1]:
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.{RegexTokenizer, Tokenizer}
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.ChiSqSelector
import org.apache.spark.ml.feature.ChiSqSelectorModel
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
import org.apache.spark.ml.feature.{HashingTF, IDF, Tokenizer}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.json4s._
import org.json4s.jackson.JsonMethods._
import java.io.{ PrintWriter, File, FileOutputStream }

sc

Intitializing Scala interpreter ...

Spark Web UI available at http://captain01.os.hpc.tuwien.ac.at:9999/proxy/application_1683456198714_2387
SparkContext available as 'sc' (version = 3.2.3, master = yarn, app id = application_1683456198714_2387)
SparkSession available as 'spark'


import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.{RegexTokenizer, Tokenizer}
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.ChiSqSelector
import org.apache.spark.ml.feature.ChiSqSelectorModel
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.{CountVectorizer, CountVectorizerModel}
import org.apache.spark.ml.feature.{HashingTF, IDF, Tokenizer}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.json4s._
import org.json4s.jackson.JsonMethods._
import java.io.{PrintWriter, File, FileOutputStream}
res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@5d8787fe


In [59]:
// import org.json4s._
// import org.json4s.jackson.JsonMethods._
// import org.apache.spark._
// import java.io.{ PrintWriter, File, FileOutputStream }

// sc

import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.apache.spark._
import java.io.{PrintWriter, File, FileOutputStream}
res16: org.apache.spark.SparkContext = org.apache.spark.SparkContext@5131f2fb


I just copy pasted the data from `Part 1` and used `toDF()`.

In [2]:
val stopwords = sc.textFile("stopwods.txt").collect()
val rdd = sc.textFile("hdfs:///user/dic23_shared/amazon-reviews/full/reviews_devset.json")
val data = rdd.map{row => val json_row = parse(row)
(compact(json_row \ "category"),
 compact(json_row \ "reviewText"))}
val reviews = data.toDF("category","reviewText")

stopwords: Array[String] = Array(a, aa, able, about, above, absorbs, accord, according, accordingly, across, actually, after, afterwards, again, against, ain, album, album, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, app, appear, appreciate, appropriate, are, aren, around, as, aside, ask, asking, associated, at, available, away, awfully, b, baby, bb, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, bibs, bike, book, books, both, brief, bulbs, but, by, c, came, camera, can, cannot, cant, car, case, cause, causes, cd, certain, certainly, changes,...


The label indexer maps a *string* column of labels to an ML column of label indices. I just labeled the *val* as *Reviews1* just for my clarity.

In [3]:
val Reviews1 = new StringIndexer()
    .setInputCol("category")
    .setOutputCol("label")
    .fit(reviews)
    .transform(reviews)

Reviews1: org.apache.spark.sql.DataFrame = [category: string, reviewText: string ... 1 more field]


In [4]:
Reviews1.show()

+--------------------+--------------------+-----+
|            category|          reviewText|label|
+--------------------+--------------------+-----+
|"Patio_Lawn_and_G...|"This was a gift ...| 18.0|
|"Patio_Lawn_and_G...|"This is a very n...| 18.0|
|"Patio_Lawn_and_G...|"The metal base w...| 18.0|
|"Patio_Lawn_and_G...|"For the most par...| 18.0|
|"Patio_Lawn_and_G...|"This hose is sup...| 18.0|
|"Patio_Lawn_and_G...|"This tool works ...| 18.0|
|"Patio_Lawn_and_G...|"This product is ...| 18.0|
|"Patio_Lawn_and_G...|"I was excited to...| 18.0|
|"Patio_Lawn_and_G...|"I purchased the ...| 18.0|
|"Patio_Lawn_and_G...|"Never used a man...| 18.0|
|"Patio_Lawn_and_G...|"Good price. Good...| 18.0|
|"Patio_Lawn_and_G...|"I have owned the...| 18.0|
|"Patio_Lawn_and_G...|"I had \"won\" a ...| 18.0|
|"Patio_Lawn_and_G...|"The birds ate al...| 18.0|
|"Patio_Lawn_and_G...|"Bought last summ...| 18.0|
|"Patio_Lawn_and_G...|"I knew I had a m...| 18.0|
|"Patio_Lawn_and_G...|"I was a little w...| 18.0|


# Pipeline

Originally I add the whole code in a single block but putting each step in a single block offers better clarity.

## Tokenizer

String cleaning, tokenization, setting to lower case. Even though the pattern looks like regex, it's just the delimiters + a bunch of escapes. 

In [5]:
val tokenizer = new RegexTokenizer()
            .setInputCol("reviewText")
            .setOutputCol("terms")
            .setPattern("[\\(\\)\\[\\]\\{\\}\\.\\!\\?,;:\\+\\='\\-\\_\\\"`~#@&*\\%€$§/\\\\1234567890\t\\s+]")
            .setMinTokenLength(2)
            .setToLowercase(true)

tokenizer: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_43c7b46a0924, minTokenLength=2, gaps=true, pattern=[\(\)\[\]\{\}\.\!\?,;:\+\='\-\_\"`~#@&*\%€$§/\\1234567890	\s+], toLowercase=true


In [6]:
tokenizer

res2: org.apache.spark.ml.feature.RegexTokenizer = RegexTokenizer: uid=regexTok_43c7b46a0924, minTokenLength=2, gaps=true, pattern=[\(\)\[\]\{\}\.\!\?,;:\+\='\-\_\"`~#@&*\%€$§/\\1234567890	\s+], toLowercase=true


## Remover

Pretty straight forward, removes the stopwords.

In [7]:
val remover = new StopWordsRemover()
            .setInputCol("terms")
            .setOutputCol("terms_remover")
            .setStopWords(stopwords)

remover: org.apache.spark.ml.feature.StopWordsRemover = StopWordsRemover: uid=stopWords_d22cc7b77fcc, numStopWords=596, locale=en_US, caseSensitive=false


## CountVectorizer

In [8]:
val cv = new CountVectorizer()
            .setInputCol("terms_remover")
            .setOutputCol("features")


cv: org.apache.spark.ml.feature.CountVectorizer = cntVec_b6ee73606928


## IDF

This takes the output of the *CountVectorizer* model as input and then scales each feature.

In [9]:
val idf = new IDF()
            .setInputCol("features")
            .setOutputCol("weightedFeatures")

idf: org.apache.spark.ml.feature.IDF = idf_07276248cba9


## Selector

This is our `ChiSqSelector()`. This selected the top 2000 features according to the Chi-Squared Test.

In [10]:
val selector = new ChiSqSelector()
          .setNumTopFeatures(2000)
          .setFeaturesCol("weightedFeatures")
          .setLabelCol("label")
          .setOutputCol("chiFeatures")

selector: org.apache.spark.ml.feature.ChiSqSelector = chiSqSelector_d08dcef568af


## Finally, the pipeline

And now we put it all together and fit it on the training set.

In [11]:
val pipeline = new Pipeline()
          .setStages(Array(tokenizer,remover,cv,idf,selector))

pipeline: org.apache.spark.ml.Pipeline = pipeline_181d71923e4e


In [12]:
val model = pipeline.fit(Reviews1)

model: org.apache.spark.ml.PipelineModel = pipeline_181d71923e4e


# Finalization

Save the vocab from *CountVectorizer* and indices of the top words.

In [13]:
val vocab = model.stages(2).asInstanceOf[CountVectorizerModel].vocabulary
val indices = model.stages(4).asInstanceOf[ChiSqSelectorModel].selectedFeatures
val topwords = indices.map(vocab(_)).sorted

vocab: Array[String] = Array(great, good, love, time, work, recommend, back, easy, make, bought, made, find, buy, price, put, reading, quality, people, works, quot, years, nice, characters, long, series, lot, found, author, day, bit, feel, makes, thing, perfect, fit, end, set, loved, things, thought, music, small, hard, give, year, world, size, worth, pretty, times, sound, written, light, real, big, amazon, part, bad, highly, money, excellent, purchased, happy, high, enjoyed, problem, family, interesting, wanted, character, job, review, purchase, man, watch, days, enjoy, place, home, stars, short, writing, play, cover, top, fan, full, fine, color, side, order, wonderful, amazing, point, fact, reviews, ordered, stories, favorite, easily, needed, battery, screen, water, dvd, beautiful, fa...


## Output generation

In [14]:
val txtfile = new PrintWriter(new File( "output_ds.txt" ))
topwords.foreach( value => {txtfile.write(value + " ")})
txtfile.close()

txtfile: java.io.PrintWriter = java.io.PrintWriter@11020ad9
